Matej Buršík

In [2]:
!pip install --use-pep517 pycaret

  Using cached pycaret-3.3.2-py3-none-any.whl.metadata (17 kB)
  Using cached scipy-1.11.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyod-2.0.2-py3-none-any.whl
  Using cached imbalanced_learn-0.12.4-py3-none-any.whl.metadata (8.3 kB)
  Using cached category_encoders-2.6.4-py2.py3-none-any.whl.metadata (8.0 kB)
  Using cached lightgbm-4.5.0-py3-none-linux_x86_64.whl
  Using cached deprecation-2.1.0-py2.py3-none-any.whl.metadata (4.6 kB)
  Using cached xxhash-3.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached matplotlib-3.7.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.7 kB)
  Using cached scikit_plot-0.3.7-py3-none-any.whl.metadata (7.1 kB)
  Using cached yellowbrick-1.5-py3-none-any.whl.metadata (7.7 kB)
  Using cached kaleido-0.2.1-py2.py3-none-manylinux1_x86_64.whl.metadata (15 kB)
  Using cached sche

In [1]:
import pyarrow
import pandas as pd
from pycaret.regression import *

In [3]:
# Load
df = pd.read_parquet('../data/after_2008_crisis.parquet', engine='pyarrow')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6076289 entries, 0 to 6076288
Data columns (total 9 columns):
 #   Column            Dtype         
---  ------            -----         
 0   price             int64         
 1   date_of_transfer  datetime64[ns]
 2   property_type     object        
 3   is_new            bool          
 4   duration          object        
 5   city              object        
 6   district          object        
 7   county            object        
 8   ppdcategory_type  object        
dtypes: bool(1), datetime64[ns](1), int64(1), object(6)
memory usage: 376.7+ MB


In [4]:
# Initialize setup
exp_clf = setup(data=df, target='price', normalize=True, n_jobs = 1)

,Description,Value
0,Session id,1559
1,Target,price
2,Target type,Regression
3,Original data shape,"(6076289, 9)"
4,Transformed data shape,"(6076289, 15)"
5,Transformed train set shape,"(4253402, 15)"
6,Transformed test set shape,"(1822887, 15)"
7,Date features,1
8,Categorical features,6
9,Preprocess,True


In [5]:
models()

,Name,Reference,Turbo
ID,,,
lr,Linear Regression,sklearn.linear_model._base.LinearRegression,True
lasso,Lasso Regression,sklearn.linear_model._coordinate_descent.Lasso,True
ridge,Ridge Regression,sklearn.linear_model._ridge.Ridge,True
en,Elastic Net,sklearn.linear_model._coordinate_descent.Elast...,True
lar,Least Angle Regression,sklearn.linear_model._least_angle.Lars,True
llar,Lasso Least Angle Regression,sklearn.linear_model._least_angle.LassoLars,True
omp,Orthogonal Matching Pursuit,sklearn.linear_model._omp.OrthogonalMatchingPu...,True
br,Bayesian Ridge,sklearn.linear_model._bayes.BayesianRidge,True
ard,Automatic Relevance Determination,sklearn.linear_model._bayes.ARDRegression,False


In [7]:
# Compare models
# options:
# budget_time = 300 //in seconds
try:
    best_model = compare_models(fold=5, include=['lr', 'lasso', 'lar', 'lightgbm'])
except ValueError as e:
    print(e)
    best_model = None

print(best_model)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,45349.3531,3982647587.2788,63108.1518,0.5864,0.3638,0.4088,70.9280
lr,Linear Regression,49899.0837,4580854286.6394,67681.9441,0.5243,0.3910,0.4528,33.4720
lasso,Lasso Regression,49898.9566,4580854583.9235,67681.9463,0.5243,0.3910,0.4528,136.0320
lar,Least Angle Regression,49898.9347,4580854624.8997,67681.9466,0.5243,0.3910,0.4528,32.3420


LGBMRegressor(n_jobs=1, random_state=1559)


In [8]:
# Use Linear Regression if no valid model is found
if best_model is not None:
    tuned_model = tune_model(best_model)

    # Predict and evaluate
    predictions = predict_model(tuned_model)
    evaluate_model(tuned_model)

    # Save the model
    save_model(tuned_model, 'model')
else:
    print("No valid model found, using Linear Regression instead.")
    # Initialize regression setup
    exp_reg = setup(data=df, target='target', verbose=True, normalize=True)
    
    # Create and tune Linear Regression model
    lr_model = create_model('lr')
    tuned_lr_model = tune_model(lr_model)

    # Predict and evaluate
    predictions = predict_model(tuned_lr_model)
    evaluate_model(tuned_lr_model)

    # Save the Linear Regression model
    save_model(tuned_lr_model, 'linear_regression_model')


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,21:05:22
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.718850 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 700
[LightGBM] [Info] Number of data points in t

KeyboardInterrupt: 